In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets 
import torchtext

print(torchtext.__version__)

0.11.1


In [ ]:
# 하이퍼파라미터 정의
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")#GPU, CPU
print("다음 기기로 학습합니다:", DEVICE)

다음 기기로 학습합니다: cuda


In [ ]:
# 데이터 로딩하기
print("데이터 로딩중...")

# data.Field 설명 #
# sequential인자 : TEXT는 Sequential 데이터라 True, Lable은 비Sequential이라 False로 설정
# batch_first : Batch를 우선시 하여, Tensor 크기를 (BATCH_SIZE, 문장의 최대 길이)로 설정
# lower : 소문자 전환 인자
# # # # # # # # # #
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

#IMDB 데이터 로딩
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

# data.Field.build_vocab() 라이브러리
# 문장 내 단어와 Integer index 를 매칭시키는 단어장(vocab)을 생성 == 워드 임베딩을 위한 Vocab 생성
# <UNK> = 0, <PAD> = 1 토큰도 추가.
# min_freq : 최소 5번 이상 등장한 단어들만 사전에 담겠다는 것. 
# 5번 미만으로 등장하는 단어는 UNK라는 토큰으로 대체
TEXT.build_vocab(trainset, min_freq=5)# TEXT 데이터를 기반으로 Vocab 생성
LABEL.build_vocab(trainset)# LABEL 데이터를 기반으로 Vocab 생성


# 학습용 데이터를 학습셋 80% 검증셋 20% 로 나누기
trainset, valset = trainset.split(split_ratio=0.8)
# 매 배치마다 비슷한 길이에 맞춰 줄 수 있도록 iterator 정의
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)


vocab_size = len(TEXT.vocab)
n_classes = 2 
# Positive, Negative Class가 두 개

'''
# Test #
for batch in train_iter:
    print(batch.text)
    print(batch.label)
    break
'''     

데이터 로딩중...


'\n# Test #\nfor batch in train_iter:\n    print(batch.text)\n    print(batch.label)\n    break\n'

In [ ]:
print("[학습셋]: %d [검증셋]: %d [테스트셋]: %d [단어수]: %d [클래스] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))

[학습셋]: 20000 [검증셋]: 5000 [테스트셋]: 25000 [단어수]: 46159 [클래스] 2


In [ ]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        print("Building Basic GRU model...")
        self.n_layers = n_layers # 일반적으로는 2

        #n_vocab : Vocab 안에 있는 단어의 개수, embed_dim : 임베딩 된 단어 텐서가 갖는 차원 값(dimension)
        self.embed = nn.Embedding(n_vocab, embed_dim)

        # hidden state vector의 dimension과 dropout 정의
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)

        #앞에서 정의한 하이퍼 파라미터를 넣어 GRU 정의
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        
        #Input: GRU의 hidden state vector(context), Output : Class probability vector
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        # Input data: 한 batch 내 모든 영화 평가 데이터
        
        x = self.embed(x)# 영화 평 임베딩

        h_0 = self._init_state(batch_size=x.size(0)) # 초기 hidden state vector를 zero vector로 생성
        x, _ = self.gru(x, h_0)  # [i, b, h] 출력값 :  (batch_size, 입력 x의 길이, hidden_dim)

        # h_t : Batch 내 모든 sequential hidden state vector의 제일 마지막 토큰을 내포한 (batch_size, 1, hidden_dim)형태의 텐서 추출
        # 다른 의미로 영화 리뷰 배열들을 압축한 hidden state vector
        h_t = x[:,-1,:]

        self.dropout(h_t)# dropout 설정 후, 

        # linear layer의 입력으로 주고, 각 클래스 별 결과 logit을 생성.
        logit = self.out(h_t)  # [b, h] -> [b, o]
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()


In [ ]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 (기존 1, 2)에서 0과 1로 변환
        optimizer.zero_grad()# 매번 기울기를 새로 계산하기 위해서 zero 로 초기화
        logit = model(x)#모델의 예측값 logit 계산
        loss = F.cross_entropy(logit, y)# logit과 실제 label간의 오차를 구하고 기울기 계산
        loss.backward()
        optimizer.step()


        if b % 30 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                e, b * len(x), 
                len(train_iter.dataset), 100. * b / len(train_iter), 
                loss.item()))
        

In [ ]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:# Validation 데이터셋에 대하여 
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    #전체 validation 셋에 대한 평균 loss와 accuracy를 구하는 과정
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [ ]:
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Building Basic GRU model...


In [ ]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[이폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" % (e, val_loss, val_accuracy))
    
  

Train Epoch: 1 [0/20000 (0%)]	Train Loss: 0.707375
Train Epoch: 1 [1920/20000 (10%)]	Train Loss: 0.706541
Train Epoch: 1 [3840/20000 (19%)]	Train Loss: 0.691229
Train Epoch: 1 [5760/20000 (29%)]	Train Loss: 0.702576
Train Epoch: 1 [7680/20000 (38%)]	Train Loss: 0.688980
Train Epoch: 1 [9600/20000 (48%)]	Train Loss: 0.692748
Train Epoch: 1 [11520/20000 (58%)]	Train Loss: 0.696841
Train Epoch: 1 [13440/20000 (67%)]	Train Loss: 0.731383
Train Epoch: 1 [15360/20000 (77%)]	Train Loss: 0.689197
Train Epoch: 1 [17280/20000 (86%)]	Train Loss: 0.688158
Train Epoch: 1 [19200/20000 (96%)]	Train Loss: 0.709162
[이폭: 1] 검증 오차: 0.69 | 검증 정확도:49.70
Train Epoch: 2 [0/20000 (0%)]	Train Loss: 0.696883
Train Epoch: 2 [1920/20000 (10%)]	Train Loss: 0.696512
Train Epoch: 2 [3840/20000 (19%)]	Train Loss: 0.690508
Train Epoch: 2 [5760/20000 (29%)]	Train Loss: 0.698974
Train Epoch: 2 [7680/20000 (38%)]	Train Loss: 0.684536
Train Epoch: 2 [9600/20000 (48%)]	Train Loss: 0.693768
Train Epoch: 2 [11520/20000 (58%)